## Code to find intersections of ICESat-2 and ATM data

**by Allison Chartrand**

**June 2019 ICESat-2 Hackweek**


In [2]:
#IMPORT PACKAGES
import os
import glob
import pandas as pd
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
import pyproj
from scipy.spatial import distance

Code to extract single tracks of IceSat-2 data

In [23]:
ATL06data = pd.read_csv('../data_prod/ZachISatData_wSmoooth.csv',parse_dates=[4])

In [24]:
ATL06data['track'].unique()

array(['gt3r', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l'], dtype=object)

In [5]:
ATL06data.date.unique()

array(['2019-01-04T12:24:25.000000000', '2019-02-15T10:09:55.000000000',
       '2018-10-29T05:05:08.000000000', '2019-01-16T01:09:49.000000000',
       '2018-11-23T03:49:31.000000000', '2018-12-10T02:50:30.000000000',
       '2018-12-19T12:57:40.000000000', '2019-01-08T01:26:37.000000000',
       '2018-11-20T14:21:44.000000000', '2018-12-26T02:17:13.000000000',
       '2019-02-17T23:37:46.000000000', '2018-11-16T14:30:09.000000000',
       '2018-11-27T03:41:06.000000000', '2018-12-27T12:41:01.000000000',
       '2018-10-21T05:21:45.000000000', '2018-12-23T12:49:22.000000000',
       '2018-10-22T15:45:34.000000000', '2018-12-14T02:42:09.000000000',
       '2018-12-15T13:05:59.000000000', '2019-01-20T01:01:33.000000000',
       '2018-12-18T02:33:52.000000000', '2019-01-13T11:42:01.000000000',
       '2019-01-29T11:08:55.000000000', '2018-11-15T04:06:22.000000000',
       '2018-11-19T03:57:57.000000000', '2018-12-02T13:56:37.000000000',
       '2018-10-17T05:30:05.000000000', '2019-01-17

Define nearest neighbor algorithm

In [6]:
def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    closest_dist = np.min(distance.cdist([node], nodes,'euclidean'))
    return closest_index, closest_dist

Get target points from ATM csv and put into dataframe

In [27]:
ATMdata = pd.read_csv('~/xtrak/data_prod/ATMprof_2018_slopes.csv')

tpoints = ATMdata[['PS_x','PS_y']].copy()
tpoints = tpoints.values
ATMdata.head()

,ATM_lat,ATM_long,PS_x,PS_y,ATM_elev,dist_along,slope_NS,slope_EW
0,79.212155,334.848567,397916.203061,-1.102322e+06,891.6397,0.000000,0.002528,-0.003607
1,79.212039,334.850146,397950.884957,-1.102323e+06,891.4861,34.695014,0.003011,-0.003841
2,79.211923,334.851724,397985.547970,-1.102324e+06,891.3228,69.371314,0.003946,-0.003423
3,79.211808,334.853304,398020.212712,-1.102325e+06,891.1725,104.046285,0.004970,-0.002393
4,79.211692,334.854884,398054.914911,-1.102326e+06,891.0138,138.761317,0.005104,-0.003135


Make an empty dataframe for intersections
Get query points from ATL06 csv and put into dataframe
Loop through unique dates and ground tracks and find the closest point between each ground track and the flowline
Store intersection along-flow distance, z_ATM, z_ATL06 and other helpful data in Intersections dataframe

In [28]:
Intersections = {'dist_along':[],'ATM_elev':[],'idx_ATM':[],'z_ATL06':[],'t_ATL06':[],'idx_ATL06':[],'gt_ATL06':[]}
Intersections = pd.DataFrame(data = Intersections)
i = 0
for day in ATL06data.date.unique():

    for tr in ATL06data.track.unique():
        close_idx = []
        min_dist = []
        AddDatarow = []

        #dfTran is a single radar transect
        dfTran = ATL06data.query('date == @day & track == @tr')
        if dfTran.shape[0] == 0:
            continue
            
        
        qpoints = dfTran[['x','y']].copy()
        qpoints = qpoints.values

        
        for j in range(len(tpoints)):
            close_idxT,min_distT = closest_node(tpoints[j,:], qpoints)
            close_idx = np.append(close_idx,[close_idxT])
            min_dist = np.append(min_dist,[min_distT])
        if min(min_dist) > 1000:
            continue
        
        tpt_idx = np.argmin(min_dist)
        qpt_idx = int(close_idx[tpt_idx])
        dfTran_idx = dfTran.loc[dfTran['date'] == day].index[0]
        qpt_idx = qpt_idx+dfTran_idx
        
        AddDatarow = {'dist_along':[ATMdata.loc[tpt_idx,'dist_along']],
                      'ATM_elev':[ATMdata.loc[tpt_idx,'ATM_elev']],
                      'idx_ATM':[tpt_idx],
                      'z_ATL06':[dfTran.loc[qpt_idx,'hSupSm']],
                      't_ATL06':[dfTran.loc[qpt_idx,'date']],
                      'idx_ATL06':[qpt_idx],
                      'gt_ATL06':[dfTran.loc[qpt_idx,'track']]}
        AddDatarow = pd.DataFrame(data = AddDatarow)
        Intersections = Intersections.append(AddDatarow,ignore_index=True)
#         Intersections.reset_index(drop=True)
        

In [9]:
Intersections.head()


,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,125289.795232,36.8164,3213.0,35.544117,2019-02-15 10:09:55,1576.0,gt1l
1,125354.749926,36.0535,3215.0,33.669106,2019-02-15 10:09:55,4931.0,gt1r
2,96208.910678,72.8016,2367.0,66.120445,2019-01-16 01:09:49,39474.0,gt3r
3,102696.987249,27.0040,2566.0,36.660946,2019-01-16 01:09:49,19999.0,gt1l
4,102632.054028,26.9541,2564.0,38.917492,2019-01-16 01:09:49,23921.0,gt1r


In [10]:
%matplotlib widget

In [29]:

f, ax = plt.subplots()
plt.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'],c=Intersections['t_ATL06'])
# ax.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'],Intersections['t_ATL06'])
# ax.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'], c = Time,s=1)
plt.plot(Intersections['dist_along']/1000,Intersections['ATM_elev'],'.')
# cb = f.colorbar()
plt.colorbar()

# cb.ax.set_yticklabels(df.index.strftime('%b %Y'))

FigureCanvasNbAgg()

In [97]:
plt.close('all')

Save to file

In [30]:
IntersectionsSort = Intersections.sort_values(by=['t_ATL06', 'gt_ATL06'])
IntersectionsSort.reset_index()

,index,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,160,122622.959615,44.7680,3314.0,42.769034,2018-10-18 15:53:52,617767.0,gt1l
1,161,122685.347575,42.9764,3316.0,42.016131,2018-10-18 15:53:52,621070.0,gt1r
2,162,125829.732109,36.7893,3416.0,34.973022,2018-10-18 15:53:52,624014.0,gt2l
3,163,125892.855741,33.7749,3418.0,32.990604,2018-10-18 15:53:52,626689.0,gt2r
4,51,80616.609191,275.4203,2136.0,NaN,2018-10-21 05:21:45,205271.0,gt2r
5,50,77393.279691,251.8895,2036.0,255.108995,2018-10-21 05:21:45,205803.0,gt3r
6,102,67676.282654,393.7867,1775.0,361.948000,2018-10-25 05:13:27,398282.0,gt1l
7,103,67709.968188,390.8599,1776.0,388.703595,2018-10-25 05:13:27,400495.0,gt1r
8,104,64585.038715,408.5852,1708.0,NaN,2018-10-25 05:13:27,401953.0,gt2l
9,105,64513.127274,411.2010,1706.0,410.961600,2018-10-25 05:13:27,403761.0,gt2r


In [31]:
InterX = IntersectionsSort.values
InterX = InterX

Dist = np.diff(InterX[:,0])
Dist[Dist == 0] = float('NaN')
InterXslopeATL06 = np.diff(InterX[:,3])/Dist
InterXslopeATM = np.diff(InterX[:,1])/Dist

# i = 0
# for i in Dist():
InterXslopeATL06[np.abs(Dist) > 100] = float('NaN')
InterXslopeATL06 = np.append(InterXslopeATL06,[0])

InterXslopeATM[np.abs(Dist) > 100] = float('NaN')
InterXslopeATM = np.append(InterXslopeATM,[float('NaN')])

print(Dist[0:10])
print(InterX.shape)
print(InterXslopeATM.shape)
print(InterXslopeATM[0:10])





[62.38796026330965 3144.384534390396 63.123631396039855 -45276.24654985474
 -3223.329499746498 -9716.99703691926 33.68553329106362 -3124.929472196309
 -71.91144090442685 -3117.049766594675]
(188, 7)
(188,)
[-0.02871707926398809 nan -0.04775390663264514 nan nan nan
 -0.08688596302485911 nan -0.03637529671358526 nan]


In [32]:
IntersectionsSort = IntersectionsSort.assign(slope_ATM= InterXslopeATM)

In [33]:
IntersectionsSort = IntersectionsSort.assign(slope_ATL06 = InterXslopeATL06)
IntersectionsSort.head()


,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06,slope_ATM,slope_ATL06
160,122622.959615,44.7680,3314.0,42.769034,2018-10-18 15:53:52,617767.0,gt1l,-0.0287171,-0.0120681
161,122685.347575,42.9764,3316.0,42.016131,2018-10-18 15:53:52,621070.0,gt1r,NaN,NaN
162,125829.732109,36.7893,3416.0,34.973022,2018-10-18 15:53:52,624014.0,gt2l,-0.0477539,-0.0314053
163,125892.855741,33.7749,3418.0,32.990604,2018-10-18 15:53:52,626689.0,gt2r,NaN,NaN
51,80616.609191,275.4203,2136.0,NaN,2018-10-21 05:21:45,205271.0,gt2r,NaN,NaN


In [21]:

IntersectionsSort.to_csv('~/xtrak/data_prod/Intersections_ATM20140429_wSlope.csv',index=False)